# Battleship Solitaire'i lahendaja

## Idee
Tegemist on lihtsa tehisintellektiga, mis lahendab pildi pealt Battleship Solitaire'i. Selle kasutamiseks lae üles pilt ning tehisintellekt lahendab selle su eest ära

## Autorid
Oliver Savolainen, Märt Tender, Kaur Vali

## Kasutatud funktsionaalsed blokid
- Pildituvastus, et selgitada ära hetkesed positsioonid.
- Lihtne reegite alusel intellekt, mis lahendab tabeli seisu.

## Pildituvastus osa

In [ ]:
## Importid piltide jaoks
import cv2 as cv
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pytesseract
#kasutame pytesseracti libraryt numbrite äratundmiseks
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract'

In [ ]:
#Loe pilt
img_rgb = cv.imread('images/puzzle6.PNG')
# Mustvalge igaks juhuks
img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)

plt.figure(figsize=(4,4))
plt.imshow(img_rgb)

In [ ]:
# meetod matchTemplate kasutades ruutude asukohtade leidmiseks
# puzzle_rgb-pilt, folder - kaust(olenevalt suuruses images kausta sees olev kaust), templateFile - template mida kasutatakse
# allFound - järjend, kus kõik asukohad kirjas
def findTemplate(puzzle_rgb,folder,templateFile,allFound):
    template = cv.imread('images/' + folder + templateFile + '.PNG')
    templates = []
    #lisame template 4 erinevat pidi, et kõikvõimalikud leitakse
    templates.append(template)
    templates.append(cv.flip(template, 1))
    templates.append(cv.flip(template, -1))
    templates.append(cv.flip(template, 0))
    w, h = template.shape[::-2]
    for i in range(len(templates)):
        res = cv.matchTemplate(puzzle_rgb,templates[i], cv.TM_CCOEFF_NORMED)
        #0.82 hetkel maksimaalne threshold, mis leiab pea alati mingi versiooni lauast
        # kõrgema korral enamasti mingeid ruutusid ei leia
        threshold = 0.82
        loc = np.where(res >= threshold)
        for pt in zip(*loc[::-1]):
            alreadyThere = False
            for found in allFound:
                #iga leitud asukoha korral kontrollime ega tegemist pole juba leitud ruuduga
                if abs(found[0] - pt[0]) < 10 and abs(found[1]-pt[1]) < 10:
                    alreadyThere = True
                    break
            if not alreadyThere:
                #kui pole varem leitud ruut, lisame vastavalt nimele selle allFoundi
                #'None' template ehk hetkel tühi ruut on None tüüpi
                if templateFile == 'None':
                    allFound.append((pt[0],pt[1],None))
                #2 korral ehk poolringi taolisel ruudul on 4 erinevat varianti, nii et anname vastavalt järjekorrale õige variandi
                elif templateFile == '2':
                    allFound.append((pt[0],pt[1],int(templateFile)*(i+1)))
                else:
                    allFound.append((pt[0],pt[1],int(templateFile)))
                # rectangle(pilt, algusnurk (koordinaadid), lõppnurk, värv (BGR), joone paksus)
                cv.rectangle(puzzle_rgb, pt, (pt[0]+w, pt[1]+h), (0,255,255), 1)

    #plt.imshow(puzzle_rgb)

In [ ]:
# meetod kõikide ruutude leidmiseks ja kontrolliks kas pikkus on vastav (variandid 6x6,8x8,10x10,15x15)
def getPuzzle(img_rgb):
    allFound = []
    sizes = [6,8,10,15]
    templates = ['None','0','1','2','5']
    #käime kõik variandid läbi alustades sellest, et vaatame, kas 6 kausta templatedega on 36 ruutu, 8 omadega 64 jne
    #ja siis ka kõik muud nagu nätieks 6 templatedega on 100 ruutu
    for i in range(len(sizes)):
        for j in range(len(sizes)):
            img_copy = img_rgb.copy()
            for template in templates:
                findTemplate(img_copy,str(sizes[j])+'/',template,allFound)
            comp = i+j
            #kui oleme järjendi lõppu jõudnud selle j korral, alustame algusest
            if comp >= len(sizes):
                comp -= len(sizes)
            if len(allFound) == sizes[comp]**2:
                return allFound
            allFound = []
    return None

In [ ]:
# meetod ruutude sorteerimiseks ja seejärel rea ja veeru kaupa täpsustamiseks asukohta, et ühtlased read ja veerud saada
def fixNumbers(allFound):
    sortedSquares = sorted(allFound, key=lambda element: (element[1]))
    for i in range(len(sortedSquares)):
        if 0 < sortedSquares[i][1] - sortedSquares[i-1][1] < 20:
            sortedSquares[i] = (sortedSquares[i][0],sortedSquares[i-1][1],sortedSquares[i][2])
    sortedSquares = sorted(sortedSquares, key=lambda element: (element[0]))
    for i in range(len(sortedSquares)):
        if 0 < sortedSquares[i][0] - sortedSquares[i-1][0] < 20:
            sortedSquares[i] = (sortedSquares[i-1][0],sortedSquares[i][1],sortedSquares[i][2])
    sortedSquares = sorted(sortedSquares, key=lambda element: (element[1],element[0]))
    return sortedSquares

In [ ]:
# lõplikusse järjendite järjendisse viimine
def coordinatesToArray(allFound):
    if allFound == None:
        return
    length = len(allFound)
    correct = None.
    #leiame suuruse
    possibleOptions = [36,64,100,225]
    for i in range(len(possibleOptions)):
        if possibleOptions[i] == length:
            correct = i
            break
    if correct == None:
        return
    #fikseerime numbrid
    sortedSquares = fixNumbers(allFound)
    fullMatrix = []
    possibleRows = [6,8,10,15]
    rowLength = possibleRows[correct]
    #viime ridadesse vastava reapikkuse kaupa
    for i in range(rowLength):
        row = [t[2] for t in sortedSquares[i*rowLength:i*rowLength+rowLength]]
        fullMatrix.append(row)
    return fullMatrix

In [ ]:
# pytesseractiga numbrite leidmine
def findNumbers(img):
    h, w, c = img.shape
    #ainult numbreid 0st 9ni leidmine
    string = pytesseract.image_to_string(img,config='digits')
    """
    boxes = pytesseract.image_to_boxes(img)
    for b in boxes.splitlines():
        b = b.split(' ')
        img = cv.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
    plt.imshow(img)
    """
    #viime järjendisse
    horizontal = [int(i) for i in list(string.strip())]
    return horizontal
    

## Mängu lahendamise osa

Lahendab reeglite põhjal mängulaua

In [ ]:
# kui mingil real või veerul peab olema null laeva, siis teeme vastavalt terve rea või veeru mereks
def zeros(index, row=False):
    global field, up, side, dim
    
    if row and side[index] == 0:
        for i in range(dim):
            set_sea(i, index)
        
    elif not row and up[index] == 0:
        for i in range(dim):
            set_sea(index, i)

In [ ]:
# koordinaadil (x,y) oleva ruudu muutmine mereks juhul,
# kui sealne koht on veel täitmata ja see koht on mänguvälja piirides
def set_sea(x, y):
    global field, dim, change
    
    if 0 <= x < dim and 0 <= y < dim and field[y][x] is None:
        change = True
        field[y][x] = 0

        
# koordinaadil (x,y) oleva ruudu muutmine laevaks juhul,
# kui sealne koht on veel täitmata ja see koht on mänguvälja piirides
def set_ship(x, y):
    global field, dim, change
    
    if 0 <= x < dim and 0 <= y < dim and field[y][x] is None:
        change = True
        # arv 3 tähistab geneerilist laevatükki ehk tüki kuju on teadamta
        field[y][x] = 3

        
# muudame lavatükke ümbritsevad ruudud mereks ja väärtuste 2, 4, 6, 8 puhul sisestame ka kõrvaloleva laevatüki
def sides_sea(x, y):
    global field, dim
    
    point = field[y][x]
    
    if point:
        # nurgad on meri sõltumata tüki kujust
        corners = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        for corr_x, corr_y in corners:
            set_sea(x + corr_x, y + corr_y)
        
        # millised küljed on meri sõltub laevatüki kujust
        if point == 1:
            set_sea(x, y - 1)
            set_sea(x - 1, y)
            set_sea(x + 1, y)
            set_sea(x, y + 1)
        elif point == 2:
            set_sea(x, y - 1)
            set_ship(x, y + 1)
        elif point == 4:
            set_sea(x - 1, y)
            set_ship(x + 1, y)
        elif point == 6:
            set_sea(x + 1, y)
            set_ship(x - 1, y)
        elif point == 8:
            set_sea(x, y + 1)
            set_ship(x, y - 1)

In [ ]:
# loe ära mitu laevatükki on vastavas reas või veerus
def count_ships(index, row=False):
    global field, dim
    
    count = 0
    
    if row:
        for i in range(dim):
            if field[index][i] not in [None, 0]:
                count += 1
    else:
        for i in range(dim):
            if field[i][index] not in [None, 0]:
                count += 1
                
    return count


# määra kõik tüjad kohad reas või veerus mereks
def all_sea(index, row=False):
    global field, up, side, dim
    
    ship_count = count_ships(index, row)
    
    if row:
        if ship_count == side[index]:
            for i in range(dim):
                set_sea(i, index)
    else:
        if ship_count == up[index]:
            for i in range(dim):
                set_sea(index, i)

In [ ]:
# loe ära mitu meretükki on vastavas reas või veerus
def count_sea(index, row=False):
    global field, dim
    
    count = 0
    
    if row:
        count = field[index].count(0)
    else:
        for i in range(dim):
            if field[i][index] == 0:
                count += 1
                
    return count
        

# määra kõik tüjad kohad reas või veerus laevadeks
def all_ship(index, row=False):
    global field, up, side, dim
    
    sea_count = count_sea(index, row)
    
    if row:
        if sea_count == dim - side[index]:
            for i in range(dim):
                set_ship(i, index)
    else:
        if sea_count == dim - up[index]:
            for i in range(dim):
                set_ship(index, i)

In [ ]:
# kõntrolli kas koordinaat (x,y) on mänguväljalt maas OR kas seal asub meri
def sea_or_off(x, y):
    global field, dim
    
    return x < 0 or x >= dim or y < 0 or y >= dim or field[y][x] == 0


# meetod, et juhul kui koordinaadil (x,y) asub laevatükk väärtusega 5, 
# siis kuidaspidiselt seda sisaldav laev mänguväljal paikneb
# ning vastavalt paigutab kõrvalolevad laevatükid
def fives(x, y):
    global field, dim
    
    point = field[y][x]
    
    if point == 5:
        if sea_or_off(x + 1, y) or sea_or_off(x - 1, y):
            set_ship(x, y + 1)
            set_ship(x, y - 1)
        elif sea_or_off(x, y + 1) or sea_or_off(x, y - 1):
            set_ship(x + 1, y)
            set_ship(x - 1, y)

In [ ]:
# kas koordinaadil (x,y) asub laevatükk
def check_ship(x, y):
    global field, dim
    
    return 0 <= x < dim and 0 <= y < dim and field[y][x] != 0


# abimeetod, et ilma erindit põhjustamata listist elemente eemaldada
def remove_if(arr, el):
    if el in arr:
        arr.remove(el)

        
# juhul kui koordinaadil (x,y) asub laevatükk väärtusega 3, siis võimaluse korral selgita välja laevatüki tegelik kuju
def fix(x, y):
    global field, dim, change
    
    if field[y][x] != 3:
        return
    
    surrounding = [
        (-1, -1), (-1, 0), (-1, 1), 
        (0, -1),  (0, 0),  (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    # kontrollime et kõik ümbritsevad ruudud oleksid juba teada (ehk pole None), 
    # sest ainult siis on laevatüki kuju võimalik kindlalt teada
    for corr_x, corr_y in surrounding:
        if 0 <= y + corr_y < dim and 0 <= x + corr_x < dim and field[y + corr_y][x + corr_x] is None:
            return
        
    poss = [1, 2, 4, 6, 8]
    
    if check_ship(x, y - 1) and check_ship(x, y + 1) or check_ship(x - 1, y) and check_ship(x + 1, y):
        field[y][x] = 5
        return
    
    if check_ship(x, y - 1):
        remove_if(poss, 1)
        remove_if(poss, 2)
        remove_if(poss, 4)
        remove_if(poss, 6)
    if check_ship(x, y + 1):
        remove_if(poss, 1)
        remove_if(poss, 4)
        remove_if(poss, 6)
        remove_if(poss, 8)
    if check_ship(x - 1, y):
        remove_if(poss, 1)
        remove_if(poss, 2)
        remove_if(poss, 4)
        remove_if(poss, 8)
    if check_ship(x + 1, y):
        remove_if(poss, 1)
        remove_if(poss, 2)
        remove_if(poss, 6)
        remove_if(poss, 8)
    
    if sea_or_off(x, y - 1):
        remove_if(poss, 8)
    if sea_or_off(x, y + 1):
        remove_if(poss, 2)
    if sea_or_off(x - 1, y):
        remove_if(poss, 6)
    if sea_or_off(x + 1, y):
        remove_if(poss, 4)
        
    if len(poss) != 1:
        # seda ei tohiks juhtuda
        print(poss)
        print(x, y)
        print_field()
        raise Exception("Võimalikke laevatüki kujusid on rohkem kui üks")
    
    field[y][x] = poss[0]

In [ ]:
# mitu iga kujuga laeva on veel mänguväljale paigutamata
def ships_remaining():
    global field, dim
    
    ships = [s for s in ships_goal]
    
    for y in range(dim):
        for x in range(dim):
            point = field[y][x]
            
            if point == 1:
                ships[0] -= 1
            elif point == 2:
                size = 1 # actually size - 1
                while field[y + size][x] == 5:
                    size += 1
                    
                if field[y + size][x] == 8:
                    ships[size] -= 1
            elif point == 4:
                size = 1 # actually size - 1
                while field[y][x + size] == 5:
                    size += 1
                    
                if field[y][x + size] == 6:
                    ships[size] -= 1
                
    return ships

In [ ]:
# mitmesse kohta mänguväljal mahuks antud suurusega laev
# mõistlik välja kutsuda ainult kõige suurema paigutamata laeva pikkusega
def fits_size(size):
    global field, dim
    
    total = 0
    
    for y in range(dim):
        c_none_1 = False
        length_1 = 0
        c_none_2 = False
        length_2 = 0
        for x in range(dim):
            if field[y][x] != 0:
                if field[y][x] is None:
                    c_none_1 = True
                length_1 += 1
            else:
                if length_1 >= size and c_none_1:
                    total += 1
                length_1 = 0
                c_none_1 = False
                
                
            if field[x][y] != 0:
                if field[x][y] is None:
                    c_none_2 = True
                length_2 += 1
            else:
                if length_2 >= size and c_none_2:
                    total += 1
                length_2 = 0
                c_none_2 = False
                
        if length_1 >= size and c_none_1:
            total += 1
        if length_2 >= size and c_none_2:
            total += 1
                
    return total

In [ ]:
# abimeetod, fill_largest() meetodis kasutamiseks
# fill_largest() leiab koha, kuhu laev tuleb panna ja siis juhul, kui laev on horisontaalne, 
# kutsub välja käesoleva meetodi, et laev ja võimaluse korral otsdes olev meri, päriselt mälnguväljale panna
def fill_horizontal(c_x, c_y, size):
    global field, dim, change
    
    spot_len = c_x[1] - c_x[0] + 1
                    
    empty = spot_len - size

    for i_x in range(c_x[0] + empty, c_x[1] - empty + 1):
        field[c_y][i_x] = 3
        change = True

    start_x = -1
    ship_len = 0
    for i_x in range(c_x[0], c_x[1] + 1):
        if field[c_y][i_x] is None:
            start_x = -1
        else:
            if start_x == -1:
                start_x = i_x
                ship_len = 0
            ship_len += 1

            if ship_len == size:
                if start_x != 0:
                    field[c_y][start_x - 1] = 0
                    change = True
                if i_x + 1 != dim:
                    field[c_y][i_x + 1] = 0
                    change = True
                break

# abimeetod, fill_largest() meetodis kasutamiseks
# fill_largest() leiab koha, kuhu laev tuleb panna ja siis juhul, kui laev on vertikaalne, 
# kutsub välja käesoleva meetodi, et laev ja võimaluse korral otsdes olev meri, päriselt mälnguväljale panna
def fill_vertical(c_x, c_y, size):
    global field, dim, change
    
    spot_len = c_y[1] - c_y[0] + 1
                    
    empty = spot_len - size

    for i_y in range(c_y[0] + empty, c_y[1] - empty + 1):
        field[i_y][c_x] = 3
        change = True

    start_y = -1
    ship_len = 0
    for i_y in range(c_y[0], c_y[1] + 1):
        if field[i_y][c_x] is None:
            start_y = -1
        else:
            if start_y == -1:
                start_y = i_y
                ship_len = 0
            ship_len += 1

            if ship_len == size:
                if start_y != 0:
                    field[start_y - 1][c_x] = 0
                    change = True
                if i_y + 1 != dim:
                    field[i_y + 1][c_x] = 0
                    change = True
                break

                
# pane antud suurusega laev kõikidesse kohtadesse kuhu see veel mahub
# võib välja kutsuda ainult suurima paigutamata laeva suurusega ja ainult,
# kui seda suurmimat laeva on veel panemata sama palju kui on kohti, kuhu sellise pikkusega laev mahub
def fill_largest(size):
    global field, dim
    
    for y in range(dim):
        c_none_1 = False
        length_1 = 0
        c_none_2 = False
        length_2 = 0
        for x in range(dim):
            if field[y][x] != 0:
                if field[y][x] is None:
                    c_none_1 = True
                length_1 += 1
            else:
                if length_1 >= size and c_none_1:
                    c_y = y
                    c_x = (x - length_1, x - 1)
                    
                    fill_horizontal(c_x, c_y, size)
                    
                length_1 = 0
                c_none_1 = False
                
                
            if field[x][y] != 0:
                if field[x][y] is None:
                    c_none_2 = True
                length_2 += 1
            else:
                if length_2 >= size and c_none_2:
                    c_y = (x - length_2, x - 1)
                    c_x = y
                    
                    fill_vertical(c_x, c_y, size)
                    
                length_2 = 0
                c_none_2 = False
                
        if length_1 >= size and c_none_1:
            c_y = y
            c_x = (dim - 1 - length_1, dim - 1)

            fill_horizontal(c_x, c_y, size)
            
        if length_2 >= size and c_none_2:
            c_y = (dim - 1 - length_2, dim - 1)
            c_x = y

            fill_vertical(c_x, c_y, size)

In [ ]:
# abimeetod väljaku printimiseks, kusjuures loetavuse huvides asendatakse None-id 7-tega
def print_field():
    global field
    
    for row in field:
        print([(x if x is not None else 7) for x in row])
    print("\n")
    

# reeglipõhise lahendamise põhimeetod
def rules():
    global change, dim, largest

    for y in range(dim):
        zeros(y)
        zeros(y, True)
    
    while True:
        change = False

        # põhireeglite rakendamine
        # osad reeglid käivad ridade ja veergude kohta, osad konkreetsete ruutude kohta
        for y in range(dim):
            all_sea(y)
            all_sea(y, True)
            all_ship(y)
            all_ship(y, True)

            for x in range(dim):
                fives(x, y)
                sides_sea(x, y)
                fix(x, y)
        
        largest_left = -1
        ships_left = ships_remaining()
        for i in range(largest):
            if ships_left[largest - 1 - i] != 0:
                largest_left = largest - i
                break
        
        if largest_left == -1:
            # siia jõudmine tähendab, et kõiki laevu on veel paigutada null tükki
            break
        
        largest_fits = fits_size(largest_left)
        
        # selle tingimuse kohta loe meetodi fill_largest() juurest
        if ships_left[largest_left - 1] == largest_fits:
            fill_largest(largest_left)
        
        if not change:
            # siia jõudmine tähendab, et laevu küll peaks veel paigutama, 
            # kuid kuna viimasel tsükli iteratsioonil ühtegi muutust ei tohimunud, 
            # ei oska ülejänud laevu reeglipõhine lahendaja paigutada
            break

## Mägimine

In [ ]:
# Loe sisse pilt
img_rgb = cv.imread('images/puzzle6.PNG')

# Leia pusle
puzzle = getPuzzle(img_rgb)
puzzleArray = coordinatesToArray(puzzle)

# Leia numbrid
puzzleNumbers = findNumbers(img_rgb)

In [ ]:
# Vajalikud globaalsed väärtused mida tuleb jooksutamiseks täita

dim = 15

field = [
    [1] + [None] * 14,
    [None] * 7 + [5] + [None] * 3 + [2] + [None] * 3,
    [None] * 15,
    [None] * 2 + [8] + [None] * 12,
    [None] * 15,
    [None] * 10 + [6] + [None] * 4,
    [None] * 15,
    [None] * 15,
    [None, 5, None, 2] + [None] * 11,
    [None] * 5 + [1] + [None] * 9,
    [None] * 13 + [1] + [None],
    [None] * 9 + [5] + [None] * 5,
    [None] * 15,
    [None] * 15,
    [None] * 7 + [1] + [None] * 7
]

up = [1, 3, 2, 3, 0, 8, 1, 2, 2, 7, 1, 4, 0, 1, 0]
side = [1, 5, 2, 3, 1, 4, 1, 1, 2, 4, 3, 1, 3, 3, 1]

largest = 5

In [ ]:
# vajalikud globaalsed muutjad

# kui palju iga pikkusega laeva mänguväljal on
# laevade pikkusega 1 kogus on largest
# kõiki järgnevaid on ühe võrra vähem, kuni largest pikkuseni, milliseid laevu on 1
ships_goal = [largest - i for i in range(largest)]

# kas viimase tsükliiteratsiooni käigus tehti mänguväljal mõni muudatus
change = False

In [ ]:
## Lahendamine ja tulemuse kuvamine
rules()
print_field()